In [159]:
import pandas as pd
import numpy as np
import altair as alt

df = pd.read_csv('./data-ages-2021-11-18/CovidFaelle_Timeline.csv', sep=';')
df = df[df['Bundesland'] == 'Österreich']
df['Time'] = pd.to_datetime(df['Time'], format='%d.%m.%Y %H:%M:%S')
df = df.loc[(df['Time'] >= '2021-09-20')]

df_hosp = pd.read_csv('./data-ages-2021-03-30-224501/CovidFallzahlen.csv', sep=';')
df_hosp = df_hosp[df_hosp['Bundesland'] == 'Alle']
df_hosp['Time'] = pd.to_datetime(df_hosp['MeldeDatum'], format='%d.%m.%Y %H:%M:%S')


# df = pd.merge(df, df_hosp, on='Time', how='inner')
df['Akut'] = df["AnzahlFaelleSum"] - df["AnzahlGeheiltSum"]
df['AnzahlFaelle7TageRolling'] = df['AnzahlFaelle'].rolling(7).mean()
df['AnzahlTotTaeglich7TageRolling'] = df['AnzahlTotTaeglich'].rolling(7).mean()
df = df.iloc[7:-7]

c_faelle = 'AnzahlFaelle7TageRolling'
c_tot = 'AnzahlTotTaeglich7TageRolling'

df.tail(10)

,Time,Bundesland,BundeslandID,AnzEinwohner,AnzahlFaelle,AnzahlFaelleSum,AnzahlFaelle7Tage,SiebenTageInzidenzFaelle,AnzahlTotTaeglich,AnzahlTotSum,AnzahlGeheiltTaeglich,AnzahlGeheiltSum,Akut,AnzahlFaelle7TageRolling,AnzahlTotTaeglich7TageRolling
6139,2021-10-31,Österreich,10,8932664,4812,832705,34675,"388,1821",11,11125,2775,766763,65942,4953.571429,11.285714
6149,2021-11-01,Österreich,10,8932664,5310,838015,36531,"408,9597",15,11140,3249,770012,68003,5218.714286,12.000000
6159,2021-11-02,Österreich,10,8932664,6601,844616,38642,"432,5921",17,11157,3899,773911,70705,5520.285714,12.142857
6169,2021-11-03,Österreich,10,8932664,9016,853632,43339,"485,1744",21,11178,3347,777258,76374,6191.285714,13.142857
6179,2021-11-04,Österreich,10,8932664,9845,863477,47226,"528,6888",17,11195,3510,780768,82709,6746.571429,14.428571
6189,2021-11-05,Österreich,10,8932664,9760,873237,50987,"570,7928",23,11218,3979,784747,88490,7283.857143,16.714286
6199,2021-11-06,Österreich,10,8932664,8173,881410,53517,"599,1158",17,11235,3982,788729,92681,7645.285714,17.285714
6209,2021-11-07,Österreich,10,8932664,8056,889466,56761,"635,4319",15,11250,4215,792944,96522,8108.714286,17.857143
6219,2021-11-08,Österreich,10,8932664,7785,897251,59236,"663,1392",24,11274,4395,797339,99912,8462.285714,19.142857
6229,2021-11-09,Österreich,10,8932664,12031,909282,64666,"723,9274",22,11296,5447,802786,106496,9238.000000,19.857143


In [160]:
def offset_column(data, x='x', y='y', offset=0):
    df_time = df['Time'].to_numpy()
    df_time_offset_x = df_time if offset==0 else  df_time[:-offset]
    df_time_offset_y = df_time if offset==0 else  df_time[offset:]

    df_data_x = df[x].to_numpy()
    df_data_x = df_data_x if offset==0 else df_data_x[:-offset]

    df_data_y = df[y].to_numpy()
    df_data_y = df_data_y if offset==0 else df_data_y[offset:]
    
    source = pd.DataFrame({
        'x': df_data_x,
        'y': df_data_y,
        'time_x': df_time_offset_x,
        'time_y': df_time_offset_y,
        'offset': offset
    })
    
    source['ratio'] = source['y'] / source['x']

    return source

In [161]:
df_offset = offset_column(df, c_faelle, c_tot, 1)
df_offset.head()

,x,y,time_x,time_y,offset,ratio
0,1705.714286,9.285714,2021-09-27,2021-09-28,1,0.005444
1,1778.714286,8.714286,2021-09-28,2021-09-29,1,0.004899
2,1791.428571,8.000000,2021-09-29,2021-09-30,1,0.004466
3,1780.714286,8.000000,2021-09-30,2021-10-01,1,0.004493
4,1778.571429,8.428571,2021-10-01,2021-10-02,1,0.004739


In [162]:
from functools import reduce

days_num = 31
dfs_offset = [offset_column(df, c_faelle, c_tot, i) for i in range(days_num)]

df_with_offset = reduce((lambda x, y: x.append(y)), dfs_offset)

df_with_offset

,x,y,time_x,time_y,offset,ratio
0,1705.714286,8.714286,2021-09-27,2021-09-27,0,0.005109
1,1778.714286,9.285714,2021-09-28,2021-09-28,0,0.005220
2,1791.428571,8.714286,2021-09-29,2021-09-29,0,0.004864
3,1780.714286,8.000000,2021-09-30,2021-09-30,0,0.004493
4,1778.571429,8.000000,2021-10-01,2021-10-01,0,0.004498
...,...,...,...,...,...,...
9,1806.428571,16.714286,2021-10-06,2021-11-05,30,0.009253
10,1822.857143,17.285714,2021-10-07,2021-11-06,30,0.009483
11,1832.857143,17.857143,2021-10-08,2021-11-07,30,0.009743
12,1836.714286,19.142857,2021-10-09,2021-11-08,30,0.010422


In [163]:
def crosscorr(datax, datay, lag=0):
    """ Lag-N cross correlation. 
    Parameters
    ----------
    lag : int, default 0
    datax, datay : pandas.Series objects of equal length

    Returns
    ----------
    crosscorr : float
    """
    return datax.corr(datay.shift(lag))

In [164]:
alt.data_transformers.disable_max_rows()

timeline_height=105

# Labels
x_label = 'aktuell positive Fälle'
y_label = 'Todesfälle'
ratio_label = '% Todesfälle/Fälle'
offset_days = 'Zeitversatz in Tagen'

# Slider + Selector
slider = alt.binding_range(min=0, max=(days_num-1), step=1, name=offset_days)
selector = alt.selection_single(
    name='OffsetSelector',
    fields=['offset'],
    bind=slider,
    init={'offset': 0}
)

# Base chart for connected scatterplot, ratio timeline and cases/icu timeline
chart_base = alt.Chart(df_with_offset)

# Connected scatterplot
connected_scatter = chart_base.add_selection(
    selector
).mark_line(
    opacity=0.3,
    point=True
).encode(
    alt.X('x:Q', axis=alt.Axis(title=x_label), scale=alt.Scale(zero=True)),
    alt.Y('y:Q', axis=alt.Axis(title=y_label), scale=alt.Scale(zero=True)),
    order='time_x:T',
    tooltip=['x:Q', 'time_x:T', 'y:Q', 'time_y:T']
).transform_filter(selector).properties(width=400,height=400)


# Ratio timeline
ratio_timeline = chart_base.mark_line().encode(
    alt.X('time_x:T', axis=alt.Axis(title='')),
    alt.Y('ratio:Q', axis=alt.Axis(format='%', title=ratio_label), scale=alt.Scale(domain=(0, .1))),
    tooltip=['time_x:T', alt.Tooltip('ratio', format='.2%')]
).transform_filter(selector).properties(width=400,height=timeline_height)

# Cases/ICU timeline
cases_timeline = chart_base.mark_line().encode(
    alt.X('time_x:T', axis=alt.Axis(title='')),
    alt.Y('x:Q', axis=alt.Axis(title='aktuell positive Faelle')),
    tooltip=['time_x:T', 'x:Q']
).transform_filter(selector).properties(width=400,height=timeline_height)
icu_timeline = chart_base.mark_line().encode(
    alt.X('time_x:T', axis=alt.Axis(title='')),
    alt.Y('y:Q', axis=alt.Axis(title='Todesfälle')),
    tooltip=['time_x:T', 'y:Q']
).transform_filter(selector).properties(width=400,height=timeline_height)

# Crosscorrelation chart
days = np.arange(days_num)
xcov = [crosscorr(df[c_faelle], df[c_tot], lag=-i) for i in range(days_num)]

source_xcov = pd.DataFrame({
    'days': days,
    'cross_correlation': xcov
})

chart_xcov = alt.Chart(source_xcov).add_selection(
    selector
).mark_bar().encode(
    alt.X('days', axis=alt.Axis(title=offset_days), scale=alt.Scale(domain=(0, days_num-1))), 
    y='cross_correlation',
    color=alt.condition('datum.days == OffsetSelector.offset', alt.value('steelblue'), alt.value('lightgray')),
    tooltip=['days', 'cross_correlation']
).properties(width=866, height=150)

# Chart composition
((connected_scatter | (cases_timeline & icu_timeline & ratio_timeline)) & chart_xcov).properties(
    title={
      "text": ["Korrelation von aktuell positiven COVID-19 Fällen und Todesfälle"], 
    }
)

alt.VConcatChart(...)

In [72]:
# calculate the covariance between two variables
from numpy.random import randn
from numpy.random import seed
from numpy import cov
# seed random number generator
seed(1)
# prepare data
df_data1 = df[c_faelle].to_numpy()
df_data1 = df_data1[:-16]

df_data2 = df[c_tot].to_numpy()
df_data2 = df_data2[16:]
data1 = df_data1
data2 = df_data2
# calculate covariance matrix
covariance = cov(data1, data2)
print(covariance)

[[1.72495644e+05 1.10070839e+03]
 [1.10070839e+03 9.00775926e+00]]


In [73]:
# calculate the Pearson's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import pearsonr
# seed random number generator
seed(1)
# prepare data
data1 = df_data1
data2 = df_data2
# calculate Pearson's correlation
corr, _ = pearsonr(data1, data2)
print('Pearsons correlation: %.3f' % corr)

Pearsons correlation: 0.883


In [74]:
# calculate the spearmans's correlation between two variables
from numpy.random import randn
from numpy.random import seed
from scipy.stats import spearmanr
# seed random number generator
seed(1)
# prepare data
data1 = df_data1
data2 = df_data2
# calculate spearman's correlation
corr, _ = spearmanr(data1, data2)
print('Spearmans correlation: %.3f' % corr)

Spearmans correlation: 0.434


In [75]:
pearson = [0.776, 0.792, 0.805, 0.821, 0.835, 0.851, 0.873, 0.892, 0.902]
spearmans = [0.686, 0.695, 0.705, 0.716, 0.718, 0.732, 0.736, 0.747, 0.749]

source_correlation_coefficient = pd.DataFrame({
    'days': [0, 1, 2, 3, 4, 5, 6, 7, 8],
    'spearmans': spearmans
})

alt.Chart(source_correlation_coefficient).mark_bar().encode(
    alt.X('days:Q'),
    alt.Y('spearmans:Q', axis=alt.Axis(format='%')),
    tooltip=['days', 'spearmans']
)

alt.Chart(...)

In [77]:
days = np.arange(60)

def f(x):
    df_data1 = df[c_faelle].to_numpy()
    
    if x > 0:
        df_data1 = df_data1[:-x]

    df_data2 = df[c_tot].to_numpy()
    df_data2 = df_data2[x:]
    
    corr, _ = pearsonr(df_data1, df_data2)
    return corr

corrs = [f(x) for x in days]

source = pd.DataFrame({
    'days': days,
    'pearsonr': corrs
})

chart_pearson = alt.Chart(source).mark_bar().encode(
    x='days',
    y='pearsonr',
    tooltip=['days', 'pearsonr']
).properties(width=800)

chart_pearson

alt.Chart(...)

In [165]:
offset_days = 16

df_time = df['Time'].to_numpy()
df_time_chart = df_time[:-offset_days]

df_data1 = df[c_faelle].to_numpy()
df_data1 = df_data1[:-offset_days]

df_data2 = df[c_tot].to_numpy()
df_data2 = df_data2[offset_days:]

cfr = pd.DataFrame({
    'infections': df_data1,
    'deaths': df_data2,
    'time': df_time_chart
})

cfr['cfr'] = cfr['deaths'] / cfr['infections']

print('CFR median: %.5f' % (cfr['cfr'].median()))
print('CFR mad: %.5f' % (cfr['cfr'].mad()))


alt.Chart(cfr).mark_bar().encode(
    alt.X('time:T'),
    alt.Y('cfr:Q', axis=alt.Axis(format='%')),
    tooltip=['time:T', alt.Tooltip('cfr:Q', format='.2%')]
).properties(width=800)

CFR median: 0.00576
CFR mad: 0.00030


alt.Chart(...)

In [84]:
days = np.arange(20)

def get_cfr(x):
    df_data1 = df[c_faelle].to_numpy()
    
    if x > 0:
        df_data1 = df_data1[:-x]

    df_data2 = df[c_tot].to_numpy()
    df_data2 = df_data2[x:]

    cfr = pd.DataFrame({
        'infections': df_data1,
        'deaths': df_data2,
    })

    cfr['cfr'] = cfr['deaths'] / cfr['infections']
    
    return cfr


def f_median(x):
    cfr = get_cfr(x)    
    return cfr['cfr'].median()

def f_mad(x):
    cfr = get_cfr(x)
    return cfr['cfr'].mad()

medians = [f_median(x) for x in days]
mads = [f_mad(x) for x in days]

source = pd.DataFrame({
    'days': days,
    'median': medians,
    'mad': mads
})

source['lower'] = source['median'] - source['mad']
source['upper'] = source['median'] + source['mad']

line = alt.Chart(source).mark_line().encode(
    x='days',
    y='median',
    tooltip=['days', 'median']
).properties(width=800)

band = alt.Chart(source).mark_area(
    opacity=0.2, color='gray'
).encode(
    x='days',
    y='lower',
    y2='upper',
    tooltip=['days', 'median', 'lower', 'upper']
).properties(width=800)

band + line

alt.LayerChart(...)

In [85]:
days_num = 65
days = np.arange(days_num)
cfr2 = get_cfr(0)
xcov = [crosscorr(df[c_faelle], df[c_tot], lag=-i) for i in range(days_num)]

source_xcov = pd.DataFrame({
    'days': days,
    'xcov': xcov
})

chart_xcov = alt.Chart(source_xcov).mark_bar().encode(
    x='days',
    y='xcov',
    tooltip=['days', 'xcov']
).properties(width=800)

chart_xcov

/Users/walterra/dev/feature-importance-demo/env/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/Users/walterra/dev/feature-importance-demo/env/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)


alt.Chart(...)

In [64]:
source

,days,median,mad,lower,upper
0,0,0.009157,0.006284,0.002873,0.015441
1,1,0.009212,0.006037,0.003175,0.015248
2,2,0.009191,0.005794,0.003397,0.014985
3,3,0.009202,0.005551,0.003651,0.014752
4,4,0.009348,0.005319,0.004029,0.014668
5,5,0.009279,0.005105,0.004174,0.014383
6,6,0.009166,0.004908,0.004258,0.014074
7,7,0.008902,0.004742,0.004160,0.013644
8,8,0.008602,0.004606,0.003996,0.013208
9,9,0.008368,0.004476,0.003892,0.012844


In [167]:
df_noma_raw = pd.DataFrame({'Time': pd.date_range(start='2021-06-01', end='2021-12-02')})

df_noma = pd.read_csv('./data-ages-2021-11-18/CovidFaelle_Timeline.csv', sep=';')
df_noma = df_noma[df_noma['Bundesland'] == 'Österreich']
df_noma['Time'] = pd.to_datetime(df_noma['Time'], format='%d.%m.%Y %H:%M:%S')
df_noma = df_noma.loc[(df_noma['Time'] >= '2021-06-01')]


df_noma['cases_calc'] = df_noma['AnzahlFaelle'] * cfr['cfr'].median()
df_noma['cases_low'] = df_noma['AnzahlFaelle'] * (cfr['cfr'].median() - cfr['cfr'].mad())
df_noma['cases_high'] = df_noma['AnzahlFaelle'] * (cfr['cfr'].median() + cfr['cfr'].mad())

df_noma = pd.merge(df_noma_raw, df_noma, how='left')

calc_offset = 16
df_noma['cases_calc'] = df_noma['cases_calc'].shift(calc_offset).ffill()
# df_noma['cases_low'] = df_noma['cases_low'].shift(calc_offset).ffill()
# df_noma['cases_high'] = df_noma['cases_high'].shift(calc_offset).ffill()

cases_calc = alt.Chart(df_noma).transform_calculate(
    name='"Todesfälle Forecast"'  
).mark_line(strokeDash=[10,2]).encode(
    alt.X('Time:T', axis=alt.Axis(title='')),
    alt.Y('cases_calc:Q', axis=alt.Axis(title='Todesfälle')),
    tooltip=['Time:T', 'cases_calc:Q'],
    color=alt.Color('name:N', scale=alt.Scale(scheme='category10'), legend=alt.Legend(title=None))
).properties(width=600,height=400)

death_real = alt.Chart(df_noma).transform_calculate(
    name='"Todesfälle"'  
).mark_line().encode(
    alt.X('Time:T', axis=alt.Axis(title='')),
    alt.Y('AnzahlTotTaeglich:Q', axis=alt.Axis(title='Todesfälle')),
    tooltip=['Time:T', 'AnzahlTotTaeglich:Q'],
    color=alt.Color('name:N', scale=alt.Scale(scheme='category10'), legend=alt.Legend(title=None))
).properties(width=600,height=400)

# band_calc = alt.Chart(df_noma).mark_area(
#     opacity=0.2, color='gray'
# ).encode(
#     x='Time:T',
#     y='cases_low',
#     y2='cases_high',
#     tooltip=['Time:T', 'cases_low', 'cases_high']
# ).properties(width=700)

cases_calc + death_real

# df_noma.tail(10)

alt.LayerChart(...)